In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

plt.style.use('seaborn-v0_8-darkgrid')

tickers = ["RELIANCE.NS", "TCS.NS", "INFY.NS"]
data = yf.download(tickers, start="2022-01-01", end="2024-12-31")
close = data["Close"]
log_returns = np.log(close / close.shift(1)).dropna()

print("Data loaded successfully!")

[*********************100%***********************]  3 of 3 completed


Data loaded successfully!


In [2]:
RISK_FREE_RATE = 0.065  # India's approx risk-free rate (6.5% annual)
TRADING_DAYS = 252

def sharpe_ratio(returns, rf=RISK_FREE_RATE):
    """
    Sharpe Ratio = (Annualized Return - Risk Free Rate) / Annualized Volatility
    Higher = better risk-adjusted return
    """
    ann_return = returns.mean() * TRADING_DAYS
    ann_vol = returns.std() * np.sqrt(TRADING_DAYS)
    return (ann_return - rf) / ann_vol

print("=" * 50)
print("Sharpe Ratio")
print("=" * 50)
for ticker in tickers:
    sr = sharpe_ratio(log_returns[ticker].dropna())
    print(f"{ticker}: {sr:.4f}")
    if sr > 1:
        print(f"  → Good risk-adjusted return")
    elif sr > 0:
        print(f"  → Positive but modest")
    else:
        print(f"  → Would have been better off in a bank!")
    print()

Sharpe Ratio
RELIANCE.NS: -0.1399
  → Would have been better off in a bank!

TCS.NS: -0.0602
  → Would have been better off in a bank!

INFY.NS: -0.1479
  → Would have been better off in a bank!



In [3]:
def compute_drawdown(price_series):
    """
    Drawdown = how far below the rolling peak we are (in %)
    This is the building block of the Ulcer Index
    """
    rolling_peak = price_series.cummax()
    drawdown = (price_series - rolling_peak) / rolling_peak
    return drawdown

In [4]:
def ulcer_index(price_series):
    """
    Ulcer Index = sqrt(mean of squared drawdowns)
    
    Why square the drawdowns?
    - Squaring penalizes BIG drawdowns much more than small ones
    - A 10% drawdown gets penalized 4x more than a 5% drawdown
    - This reflects real investor pain — big sustained losses hurt a LOT more
    
    This is the EXACT risk metric used in Dr. Jindal's FOUI-LR paper
    """
    drawdown = compute_drawdown(price_series)
    squared_drawdowns = drawdown ** 2
    ui = np.sqrt(squared_drawdowns.mean())
    return ui

print("=" * 50)
print("Ulcer Index")
print("=" * 50)
for ticker in tickers:
    ui = ulcer_index(close[ticker].dropna())
    print(f"{ticker}: {ui:.4f}")
    print(f"  → Lower = less painful drawdowns = less investor stress")
    print()

Ulcer Index
RELIANCE.NS: 0.1024
  → Lower = less painful drawdowns = less investor stress

TCS.NS: 0.1241
  → Lower = less painful drawdowns = less investor stress

INFY.NS: 0.1947
  → Lower = less painful drawdowns = less investor stress

